In [1]:
# Have to patch sys.path until we have tighter taucmdr/anaconda integration.
# Need to move towards taucmdr as a conda package.
import os
import sys
try:
    import taucmdr
except ImportError:
    sys.path.insert(0, os.path.join(os.environ['__TAUCMDR_HOME__'], 'packages'))
finally:
    from taucmdr.model.project import Project

In [2]:
%pylab
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import pandas as pd
import math
import numpy as np
import operator
import time
import re
#import mpld3
#mpld3.enable_notebook()
#from IPython.display import JSON

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
num_trials = Project.selected().experiment().num_trials
trials = Project.selected().experiment().trials(xrange(0, num_trials))
trial_data = {}
for i in xrange(0, num_trials):
    trial_data[i] = trials[i].get_data()

In [4]:
#trial0_intervals = trial_data[0][0][0][0].interval_data()
#trial0_intervals.plot(kind='bar')
#plt.show()

In [5]:
start = time.time()
trial_intervals = []
for trial in xrange(0, num_trials):
    for i in xrange(0, len(trial_data[trial])):
        for j in xrange(0, len(trial_data[trial][i])):
            for k in xrange(0, len(trial_data[trial][i][j])):
                trial_intervals.append(trial_data[trial][i][j][k].interval_data())
                #x = trial_data[trial][i][j][k].interval_data()
                #x['percentage'] = x['Exclusive']/x.loc['.TAU application', 'Inclusive']
                #trial_intervals.append(x)
                
expr_intervals = pd.concat(trial_intervals)
end = time.time()
print 'Time spent constructing dataframe of size %sx%s: %s' %(expr_intervals.shape[0], expr_intervals.shape[1], end-start)

Time spent constructing dataframe of size 756x6: 0.228271961212


In [6]:
# levels: 0=trial, 1=node, 2=context, 3=thread, 4=region name

def filter_regions(dfs, percentage=0.1):
    unstacked_dfs = dfs.unstack(4)
    dfs['percentage'] = unstacked_dfs.loc[:,'Exclusive'].div(unstacked_dfs.loc[:,('Inclusive','.TAU application')], axis=0).stack()
    dfs_filtered = dfs.groupby(level=4).filter(lambda x: x['percentage'].max()>percentage or x.name == '.TAU application')
    print 'Filtering all regions with less than %s%% of total runtime reduced number of regions from %s to %s.'%(100*percentage,len(dfs.index.get_level_values(4).unique()), len(dfs_filtered.index.get_level_values(4).unique()))
    return dfs_filtered

def largest_stddev(dfs,n):
    return dfs['Exclusive'].groupby(level=3).std(ddof=0).dropna().sort_values(ascending=False, axis=0)[:n]

def largest_correlation(dfs,n):
    unstacked_dfs = dfs.unstack(4)
    return unstacked_dfs.loc[:,'Exclusive'].corrwith(unstacked_dfs.loc[:,('Inclusive','.TAU application')]).sort_values(ascending=False, axis=0)[:n]

def largest_exclusive(dfs,n):
    return dfs['Exclusive'].groupby(level=4).max().nlargest(n)

def hotspots(dfs, n, flag):
    if flag == 0:
        largest = largest_exclusive(dfs,n)
    elif flag == 1:
        largest = largest_stddev(dfs,n)
    elif flag == 2:
        largest = largest_correlation(dfs,n)
    else:
        print 'Invalid flag'
    y = ['exclusive time', 'standard deviation', 'correlation to total runtime']
    print 'Hotspot Analysis Summary'
    print '='*80
    print 'The code regions with largest %s are: ' %y[flag]
    for i in xrange(0,n):
        print '%s: %s (%s)' %(i+1, largest.index[i], largest[i])
    
n=5

# Hotspot analysis without filtering
nofiltering_start = time.time()
start = time.time()
hotspots(expr_intervals, n, 0)
end = time.time()
print '\nTime spent finding %s most expensive regions: %s\n\n' %(n, end-start)

start = time.time()
hotspots(expr_intervals.loc[0], n, 1)
end = time.time()
print '\nTime spent finding %s regions with largest standard deviation: %s\n\n' %(n, end-start)

start = time.time()
hotspots(expr_intervals, n, 2)
end = time.time()
nofiltering_end = time.time()
print '\nTime spent finding %s regions with highest correlation to total runtime: %s\n\n' %(n, end-start)

# Hotspot analysis with filtering
print '='*80

filtering_start = time.time()
start = time.time()
filtered_dfs = filter_regions(expr_intervals, 0.05)
end = time.time()
print '\nTime spent filtering the dataframe: %s\n\n' %(end-start)
filtered_dfs

start = time.time()
hotspots(filtered_dfs, n, 0)
end = time.time()
print '\nTime spent finding %s most expensive regions: %s\n\n' %(n, end-start)

start = time.time()
hotspots(filtered_dfs.loc[0], n, 1)
end = time.time()
print '\nTime spent finding %s regions with largest standard deviation: %s\n\n' %(n, end-start)

start = time.time()
hotspots(filtered_dfs, n, 2)
end = time.time()
filtering_end = time.time()
print '\nTime spent finding %s regions with highest correlation to total runtime: %s\n\n' %(n, end-start)

print 'Hotspot analysis took %s seconds without filtering and %s seconds with filtering.' %(nofiltering_end-nofiltering_start, filtering_end-filtering_start)

Hotspot Analysis Summary
The code regions with largest exclusive time are: 
1: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]   (5473132.0)
2: void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]   (5473132.0)
3: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]   (3972973.0)
4: void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]   (3972973.0)
5: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => MPI_Finalize()   (1901455.0)

Time spent finding 5 most expensive regions: 0.00324606895447


Hotspot A

In [7]:
# Generate 3D plot
#def plot3d(trial_intervals):
#    pass
#
#plot3d(expr_intervals.loc[0])

In [8]:
#Stacked 2D bar chart for experiment 1
expr_intervals.loc[1, 'Exclusive'].unstack().plot.barh(stacked=True, sort_columns=True)

#Bar chart for each function
#expr_intervals.loc[1, 'Exclusive'].plot.barh()
plt.show()

In [9]:
# Table with links
from IPython.core.display import display, HTML, display_html

_location_re = re.compile(r'\{(.*)\} {(\d+),(\d+)}-{(\d+),(\d+)}')
def parse_region(region):
    func = region.split('=>')[-1]
    loc = re.search(r'\[(.*)\]', func)
    if loc:
        location = loc.group(1)
        match = _location_re.match(location)
        return match.group(1)

def add_link(multiindex):
    link = parse_region(multiindex[4])
    if link:
        return (multiindex[0],multiindex[1],multiindex[2],multiindex[3],'<a href="{0}">{1}</a>'.format((link), multiindex[4]))
    else:
        return multiindex

expr_intervals_link = expr_intervals.copy()
expr_intervals_link.index = expr_intervals_link.index.map(lambda x: add_link(x))
HTML(expr_intervals_link.to_html(escape=False))

In [10]:
# Collapsible table
#JSON(expr_intervals.loc[(0,0,0,0)].to_json(orient='index'))